In [1]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
import pickle
import gzip
import pdb

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [2]:
train_set, valid_set, test_set = pickle.load(gzip.open('mnist.pkl.gz', 'rb'), encoding='latin1')

In [3]:
train_set = (np.vstack([train_set[0], valid_set[0]]),
             (np.hstack([train_set[1], valid_set[1]])))
srng = RandomStreams()

In [4]:
def weight_variable(shape):
    return theano.shared(np.asarray(np.random.normal(0, 0.1, shape), dtype=theano.config.floatX))

def bias(shape):
    return theano.shared(np.full(shape, 0.1, dtype=theano.config.floatX))

def conv2d(x, W):
    return theano.tensor.nnet.conv.conv2d(x, W, filter_shape=[1,5,5,32], border_mode='full')

def max_pool_2x2(x):
    return theano.tensor.signal.pool.pool_2d(x, ws=(2,2), ignore_border=True)

def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def Adam(cost, params, lr=0.001, b1=0.1, b2=0.0001, e=1e-8):
    pdb.set_trace()
    updates = []
    grads = T.grad(cost, params)
    i = theano.shared(np.full([1], 0.0, dtype=theano.config.floatX))
    i_t = i + 1.
    fix1 = 1. - (1. - b1)**i_t
    fix2 = 1. - (1. - b2)**i_t
    lr_t = lr * (T.sqrt(fix2) / fix1)
    for p, g in zip(params, grads):
        m = theano.shared(p.get_value() * 0.)
        v = theano.shared(p.get_value() * 0.)
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
        #g_t = m_t / (T.sqrt(v_t) + e)
        g_t = mt
        p_t = p - (lr_t * g_t)
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    updates.append((i, i_t))
    return updates

In [5]:
x = T.fmatrix('x')
y = T.ivector('y')

x_image = theano.tensor.reshape(x, [-1, 1,28, 28])
y_oh = theano.tensor.extra_ops.to_one_hot(y, 10)
# First conv layer

W_conv1 = weight_variable([32, 1, 5, 5])
#b_conv1 = bias(32)

h_conv1 = theano.tensor.nnet.relu(conv2d(x_image, W_conv1)[:, :, 2:28, 2:28])
h_pool1 = max_pool_2x2(h_conv1)

# Second conv layer

W_conv2 = weight_variable([64, 32, 5, 5])
#b_conv2 = bias([64])

h_conv2 = theano.tensor.nnet.relu(conv2d(h_pool1, W_conv2)[:, :, 2:16, 2:16])
h_pool2 = max_pool_2x2(h_conv2)

In [6]:
# fully connected layer
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias([1024])

h_pool2_flat = theano.tensor.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = theano.tensor.nnet.relu(theano.tensor.dot(h_pool2_flat, W_fc1)+b_fc1)

h_fc1_drop = dropout(h_fc1, 0.5)

In [7]:
# output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias([10])

y_conv = theano.tensor.nnet.softmax(theano.tensor.dot(h_fc1_drop, W_fc2)+b_fc2)
y_pred = theano.tensor.argmax(y_conv, axis=1)

In [8]:
loss = theano.tensor.mean(theano.tensor.nnet.nnet.categorical_crossentropy(y_conv, y))

#params = [W_conv1, b_conv1, W_conv2, b_conv2, W_fc1, b_fc1, W_fc2, b_fc2]
params = [W_conv1, W_conv2, W_fc1, b_fc1, W_fc2, b_fc2]
updates = Adam(loss, params, lr=1e-4)

train = theano.function(inputs=[x, y], outputs=loss, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[x], outputs=y_pred, allow_input_downcast=True)

TypeError: Variables do not support boolean operations.

In [ ]:

batch_size = 50
print("hello")
for i in range(20000):
    idxs = np.random.randint(0, train_set[0].shape[0]-1, 50)
    X_batch = train_set[0][idxs, :]
    Y_batch = train_set[1][idxs]
    loss = train(X_batch, Y_batch)
    if i%1000==0:
        accuracy = np.mean(Y_batch == predict(X_batch))
        print("Step: {}, accuracy: {:.2f}".format(i, accuracy*100))

test_accuracy = np.mean(np.argmax(test_set[1], axis=1) == predict(test_set[0]))